# Import Libs

In [1]:
import re
import string

import pandas as pd
import numpy as np
from statistics import mean

## Tokenize the comments
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer


from nltk import sent_tokenize, word_tokenize

In [2]:
# Download dependencies

#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('punkt_tab')
#nltk.download('wordnet')

# Load dataset

In [3]:
df = pd.read_csv('./dataset/2019-05-28_portuguese_hate_speech_binary_classification.csv')

In [4]:
df

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E
...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A


In [5]:
(df['hatespeech_comb'].value_counts()/len(df))*100

hatespeech_comb
0    68.465608
1    31.534392
Name: count, dtype: float64

# Pre-Processing

In [6]:
## Additional pre-processing

# remove numbers
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

# remove urls
def remove_urls(text):
    result = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return result

# remove twitter usernames
def remove_username(text):
    return re.sub(r'@([A-Za-z0-9-_]+) ', '', text)

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# tokenize
def tokenize(text):
    text = word_tokenize(text)
    return text

# remove stopwords
stop_words = set(stopwords.words('portuguese'))
def remove_stopwords(text):
    text = [i for i in text if not i in stop_words]
    return text

# lemmatize
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    text = [lemmatizer.lemmatize(token) for token in text]
    return text
 
def preprocessing(text):
    text = text.lower()
    text = remove_urls(text)
    text = remove_username(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = tokenize(text)
    text = remove_stopwords(text)
    text = lemmatize(text)
    text = ' '.join(text)
    return text

In [7]:
df['pre_processed_text'] = df['text'].apply(preprocessing)

In [8]:
df

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3,pre_processed_text
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E,cara vive outro mundo mundo real refugiados vi...
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C,incompetentes cuidam povo brasileiro poucos re...
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E,cumpanhero quebraram toda regras
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D,conseguem pensar sentido lato além vê frente o...
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E,bom dia macaco branco haha
...,...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A,unico nao corrompenao vende chega aroporto apl...
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A,sei cotas sabia disso putaria porra
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C,sim maioria eleitorado mulher
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A,seguir tt facíl apresentar amigas sapatão ngm ...


In [9]:
df.to_csv('./dataset/hsd_pre_processed.csv', index=False)

# Word Embedding

## Word2Vec - GloVe

In [10]:
GLOVE_MODEL_FILE = './dataset/glove.twitter.27B/glove.twitter.27B.100d.txt'

# Load the GloVe embeddings into a dictionary
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]  # The word
            vector = np.asarray(values[1:], dtype=float)  # The embedding vector
            embeddings_index[word] = vector
    return embeddings_index

embeddings_index = load_glove_embeddings(GLOVE_MODEL_FILE)
print(f"Loaded {len(embeddings_index)} word vectors.")

Loaded 1193514 word vectors.


In [11]:
df['text_glove'] = df['pre_processed_text'].apply(word_tokenize)

def embed_text(tokens, embeddings_index, embedding_dim=100):
    embeddings = []
    for word in tokens:
        vector = embeddings_index.get(word)
        if vector is not None:
            embeddings.append(vector)
        else:
            # Handle out-of-vocabulary (OOV) words
            embeddings.append(np.zeros(embedding_dim))
    return np.array(embeddings)

df['text_glove'] = df['text_glove'].apply(lambda text: embed_text(text, embeddings_index))

In [12]:
df

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3,pre_processed_text,text_glove
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E,cara vive outro mundo mundo real refugiados vi...,"[[-0.11667, -0.5588, 0.55324, 0.27111, -0.0864..."
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C,incompetentes cuidam povo brasileiro poucos re...,"[[0.29807, -0.87606, 0.57167, -0.15141, 0.2922..."
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E,cumpanhero quebraram toda regras,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D,conseguem pensar sentido lato além vê frente o...,"[[0.28124, -0.32116, 1.0204, 0.5795, 0.29654, ..."
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E,bom dia macaco branco haha,"[[0.36336, -0.23872, 0.12917, -0.51607, 0.2313..."
...,...,...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A,unico nao corrompenao vende chega aroporto apl...,"[[-0.43466, -0.26563, -0.17129, 0.40573, 0.371..."
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A,sei cotas sabia disso putaria porra,"[[-0.11048, -1.0303, 0.5282, 0.75377, 0.23368,..."
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C,sim maioria eleitorado mulher,"[[0.54497, -1.0734, 0.0048874, 0.061676, -0.18..."
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A,seguir tt facíl apresentar amigas sapatão ngm ...,"[[0.19611, -0.25271, 0.017712, -0.83988, 0.345..."


In [13]:
df.to_csv('./dataset/hsd_pre_processed_glove.csv', index=False)

## Gemini

In [14]:
gemini_api_key = ""

In [15]:
import google.generativeai as genai

genai.configure(api_key=gemini_api_key)

In [16]:
def get_embed(text):
    return genai.embed_content(
            model="models/text-embedding-004",
            content=text)['embedding']

In [17]:
df['gemini_embedding'] = df['pre_processed_text'].apply(lambda text: get_embed(text))

In [18]:
df

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3,pre_processed_text,text_glove,gemini_embedding
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E,cara vive outro mundo mundo real refugiados vi...,"[[-0.11667, -0.5588, 0.55324, 0.27111, -0.0864...","[-0.015683623, -0.03517303, -0.01821586, -0.00..."
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C,incompetentes cuidam povo brasileiro poucos re...,"[[0.29807, -0.87606, 0.57167, -0.15141, 0.2922...","[0.024728414, 0.013021446, -0.039535552, 0.015..."
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E,cumpanhero quebraram toda regras,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.02794479, 0.012976087, -0.06279022, -0.012..."
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D,conseguem pensar sentido lato além vê frente o...,"[[0.28124, -0.32116, 1.0204, 0.5795, 0.29654, ...","[-0.020720925, 0.055354297, -0.028093176, -0.0..."
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E,bom dia macaco branco haha,"[[0.36336, -0.23872, 0.12917, -0.51607, 0.2313...","[-0.07149509, -0.0039677643, 0.014008735, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A,unico nao corrompenao vende chega aroporto apl...,"[[-0.43466, -0.26563, -0.17129, 0.40573, 0.371...","[0.0044391938, 0.023174051, 0.020349946, -0.00..."
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A,sei cotas sabia disso putaria porra,"[[-0.11048, -1.0303, 0.5282, 0.75377, 0.23368,...","[-0.06403098, 0.0318813, -0.017783472, -0.0311..."
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C,sim maioria eleitorado mulher,"[[0.54497, -1.0734, 0.0048874, 0.061676, -0.18...","[0.015019704, 0.018858356, 0.029902609, -0.020..."
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A,seguir tt facíl apresentar amigas sapatão ngm ...,"[[0.19611, -0.25271, 0.017712, -0.83988, 0.345...","[-0.04061569, 0.021848962, 0.013160055, -0.010..."


# Save dataset

In [19]:
df.to_csv('./dataset/hsd_pre_processed_gemini.csv', index=False)